In [2]:
import torch
%load_ext autoreload
%autoreload 2

In [3]:
from model.threedepn import ThreeDEPNDecoder
from util.model import summarize_model

threedepn = ThreeDEPNDecoder()
print(summarize_model(threedepn))

   | Name         | Type             | Params  
-----------------------------------------------------
0  | bottleneck   | Sequential       | 197376  
1  | bottleneck.0 | Linear           | 65792   
2  | bottleneck.1 | ReLU             | 0       
3  | bottleneck.2 | Linear           | 131584  
4  | bottleneck.3 | ReLU             | 0       
5  | decoder1     | Sequential       | 8389376 
6  | decoder1.0   | ConvTranspose3d  | 8388864 
7  | decoder1.1   | BatchNorm3d      | 512     
8  | decoder1.2   | ReLU             | 0       
9  | decoder2     | Sequential       | 2097536 
10 | decoder2.0   | ConvTranspose3d  | 2097280 
11 | decoder2.1   | BatchNorm3d      | 256     
12 | decoder2.2   | ReLU             | 0       
13 | decoder3     | Sequential       | 524480  
14 | decoder3.0   | ConvTranspose3d  | 524352  
15 | decoder3.1   | BatchNorm3d      | 128     
16 | decoder3.2   | ReLU             | 0       
17 | decoder4     | Sequential       | 4097    
18 | decoder4.0   | ConvTranspose3

In [4]:
from data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train', filter_class = 'sofa')
overfit_dataset = ShapeNet('overfit')

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}')  # expected output: 153540
# Get length, which is a call to __len__ function
print(f'Length of overfit set: {len(overfit_dataset)}')  # expected output: 64

Length of train set: 3173
Length of overfit set: 64


In [5]:
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = train_dataset[0]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Target DF: (32, 32, 32)


Output()

In [ ]:
##################
#                #
#    TRAINING    #
#                #
##################

In [5]:
# OVERFIT
from training import train
config = {
    'experiment_name': 'overfit',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': True,
    'batch_size': 32,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 4,
    'print_every_n': 1,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'vad_free' : True,
    'test': False,
    'kl_weight': 1,
    'resume_ckpt': None

}
train.main(config)

Using device: cuda:0
Loading saved model, latent codes, and latent variances...
Training params: 3
[001/00004] train_loss: 0.010115 kl_loss: 0.007572 normal_loss: 0.002543
[001/00004] train_loss: 0.010374 kl_loss: 0.007749 normal_loss: 0.002626
[002/00004] train_loss: 0.008716 kl_loss: 0.007542 normal_loss: 0.001174
[002/00004] train_loss: 0.008393 kl_loss: 0.007422 normal_loss: 0.000971
[003/00004] train_loss: 0.008175 kl_loss: 0.007302 normal_loss: 0.000872
[003/00004] train_loss: 0.008076 kl_loss: 0.007325 normal_loss: 0.000750
[004/00004] train_loss: 0.007750 kl_loss: 0.007125 normal_loss: 0.000625
[004/00004] train_loss: 0.007689 kl_loss: 0.007174 normal_loss: 0.000515


In [9]:
# AIRPLANE
from training import train
config = {
    'experiment_name': 'airplane_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 20,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'airplane',
    'decoder_var' : False
}
train.main(config)

Using device: cuda:0
Training params: 2
[001/00035] train_loss: 0.107308 kl_loss: 0.000000 normal_loss: 0.107308
[003/00007] train_loss: 0.045517 kl_loss: 0.000000 normal_loss: 0.045517
[004/00043] train_loss: 0.036284 kl_loss: 0.000000 normal_loss: 0.036284
[006/00015] train_loss: 0.034843 kl_loss: 0.000000 normal_loss: 0.034843
[007/00051] train_loss: 0.030958 kl_loss: 0.000000 normal_loss: 0.030958
[009/00023] train_loss: 0.029536 kl_loss: 0.000000 normal_loss: 0.029536
[010/00059] train_loss: 0.031433 kl_loss: 0.000000 normal_loss: 0.031433
[012/00031] train_loss: 0.030174 kl_loss: 0.000000 normal_loss: 0.030174
[014/00003] train_loss: 0.029374 kl_loss: 0.000000 normal_loss: 0.029374
[015/00039] train_loss: 0.028661 kl_loss: 0.000000 normal_loss: 0.028661
[017/00011] train_loss: 0.027764 kl_loss: 0.000000 normal_loss: 0.027764
[018/00047] train_loss: 0.026550 kl_loss: 0.000000 normal_loss: 0.026550
[020/00019] train_loss: 0.025417 kl_loss: 0.000000 normal_loss: 0.025417
[021/00055]

Traceback (most recent call last):
  File "c:\Users\Munzer Dwedari\miniconda3\envs\adl4cv\lib\site-packages\IPython\core\interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Munzer Dwedari\AppData\Local\Temp\ipykernel_16500\673902499.py", line 20, in <cell line: 20>
    train.main(config)
  File "c:\Users\Munzer Dwedari\Documents\uni\ADL4CV\adl4cv-vad\training\train.py", line 167, in main
  File "c:\Users\Munzer Dwedari\Documents\uni\ADL4CV\adl4cv-vad\training\train.py", line 55, in train
    for epoch in range(config['max_epochs']):
  File "c:\Users\Munzer Dwedari\miniconda3\envs\adl4cv\lib\site-packages\torch\utils\data\dataloader.py", line 368, in __iter__
    return self._get_iterator()
  File "c:\Users\Munzer Dwedari\miniconda3\envs\adl4cv\lib\site-packages\torch\utils\data\dataloader.py", line 314, in _get_iterator
    return _MultiProcessingDataLoaderIter(self)
  File "c:\Users\Munzer Dwedari\miniconda3\envs\adl4cv\l

In [89]:
# CHAIR
from training import train
config = {
    'experiment_name': 'chair_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'chair',
    'decoder_var' : False
}
train.main(config)

Using device: cuda:0
Data length 6000
Training params: 2
[001/00005] train_loss: 0.170789 kl_loss: 0.000000 normal_loss: 0.170789
[002/00011] train_loss: 0.119310 kl_loss: 0.000000 normal_loss: 0.119310
[003/00017] train_loss: 0.112543 kl_loss: 0.000000 normal_loss: 0.112543
[004/00023] train_loss: 0.106655 kl_loss: 0.000000 normal_loss: 0.106655
[005/00029] train_loss: 0.104214 kl_loss: 0.000000 normal_loss: 0.104214
[006/00035] train_loss: 0.098748 kl_loss: 0.000000 normal_loss: 0.098748
[007/00041] train_loss: 0.095269 kl_loss: 0.000000 normal_loss: 0.095269
[008/00047] train_loss: 0.090359 kl_loss: 0.000000 normal_loss: 0.090359
[009/00053] train_loss: 0.089151 kl_loss: 0.000000 normal_loss: 0.089151
[010/00059] train_loss: 0.084609 kl_loss: 0.000000 normal_loss: 0.084609
[011/00065] train_loss: 0.083053 kl_loss: 0.000000 normal_loss: 0.083053
[012/00071] train_loss: 0.081139 kl_loss: 0.000000 normal_loss: 0.081139
[013/00077] train_loss: 0.077102 kl_loss: 0.000000 normal_loss: 0.0

KeyboardInterrupt: 

In [ ]:
# SOFA & CHAIR VAD
from training import train
config = {
    'experiment_name': 'sofa_chair_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'sofa_chair',
    'decoder_var' : True
}
train.main(config)

In [163]:
# SOFA VAD
from training import train
config = {
    'experiment_name': 'sofa_vad_0_03kl',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'sofa',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Data length 3173
Training params: 3
[001/00049] train_loss: 0.158797 kl_loss: 0.485190 normal_loss: 0.144241
[003/00049] train_loss: 0.103193 kl_loss: 0.413880 normal_loss: 0.090777
[005/00049] train_loss: 0.092658 kl_loss: 0.348836 normal_loss: 0.082193
[007/00049] train_loss: 0.089486 kl_loss: 0.300002 normal_loss: 0.080486
[009/00049] train_loss: 0.084510 kl_loss: 0.266648 normal_loss: 0.076511
[011/00049] train_loss: 0.080834 kl_loss: 0.244796 normal_loss: 0.073490
[013/00049] train_loss: 0.075477 kl_loss: 0.232309 normal_loss: 0.068508
[015/00049] train_loss: 0.072623 kl_loss: 0.224983 normal_loss: 0.065873
[017/00049] train_loss: 0.070084 kl_loss: 0.216381 normal_loss: 0.063592
[019/00049] train_loss: 0.068342 kl_loss: 0.208961 normal_loss: 0.062073
[021/00049] train_loss: 0.065641 kl_loss: 0.202974 normal_loss: 0.059552
[023/00049] train_loss: 0.062942 kl_loss: 0.197744 normal_loss: 0.057010
[025/00049] train_loss: 0.061899 kl_loss: 0.192079 normal_loss: 0.0

In [48]:
# SOFA AD
from training import train
config = {
    'experiment_name': 'sofa_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'sofa',
    'decoder_var' : False
}
train.main(config)

Using device: cuda:0
Data length 3173
Training params: 2
[001/00049] train_loss: 0.147046 kl_loss: 0.000000 normal_loss: 0.147046
[003/00049] train_loss: 0.086997 kl_loss: 0.000000 normal_loss: 0.086997
[005/00049] train_loss: 0.078271 kl_loss: 0.000000 normal_loss: 0.078271
[007/00049] train_loss: 0.072125 kl_loss: 0.000000 normal_loss: 0.072125
[009/00049] train_loss: 0.063552 kl_loss: 0.000000 normal_loss: 0.063552
[011/00049] train_loss: 0.060275 kl_loss: 0.000000 normal_loss: 0.060275
[013/00049] train_loss: 0.056823 kl_loss: 0.000000 normal_loss: 0.056823
[015/00049] train_loss: 0.056097 kl_loss: 0.000000 normal_loss: 0.056097
[017/00049] train_loss: 0.051553 kl_loss: 0.000000 normal_loss: 0.051553
[019/00049] train_loss: 0.050349 kl_loss: 0.000000 normal_loss: 0.050349
[021/00049] train_loss: 0.047069 kl_loss: 0.000000 normal_loss: 0.047069
[023/00049] train_loss: 0.044120 kl_loss: 0.000000 normal_loss: 0.044120
[025/00049] train_loss: 0.043236 kl_loss: 0.000000 normal_loss: 0.0

KeyboardInterrupt: 

In [10]:
#####################
#                   #
#    VISUALIZING    #
#                   #
#####################

In [299]:
from model.threedepn import ThreeDEPNDecoder
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
import torch

def visualize_dataset_sample(filter_class, index):
    dataset = ShapeNet('train', filter_class = filter_class)
    sample = dataset[index]
    input_mesh = marching_cubes(sample['target_df'], level=1)
    visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

def visualize_ad(experiment, index):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Load latent codes
    latent_vectors = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    # Sample
    x = latent_vectors[index].unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

def visualize_vad(experiment, index):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Load latent codes
    latent_vectors = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    log_vars = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    # Sample
    x = latent_vectors[index]
    Dist = dist.Normal(x, torch.exp(log_vars[index]))
    x_vad = Dist.rsample().unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x_vad)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

def visualize_vad_norm(experiment):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Sample
    # Dist = dist.Normal(torch.zeros(256), torch.ones(256))
    x_vad = torch.randn(256).unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x_vad)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

def visualize_interpolation_ad(experiment, index1, index2, a1=0.5, a2=0.5):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Load latent codes
    latent_vectors = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    # Sample
    x1 = latent_vectors[index1]
    x2 = latent_vectors[index2]
    x = (a1*x1 + a2*x2).unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

In [314]:
index = 0
index1 = 15
index2 = 2334
a1 = 0.5
a2 = 1 - a1
# experiment = "sofa_ad"
# filter_class = "sofa"
experiment = "sofa_vad_0_03kl"
experiment2 = "sofa_ad"
filter_class = "sofa"
visualize_vad_norm(experiment)
visualize_vad_norm(experiment2)
#-------
# visualize_vad(experiment, index)
# visualize_ad(experiment, index)
# visualize_dataset_sample(filter_class, index)
#-------
# visualize_interpolation_ad("chair_ad", index1, index2, a1, a2)
# visualize_ad(experiment, index1)
# visualize_ad(experiment, index2)

Output()

Output()

In [11]:
from model.threedepn import ThreeDEPNDecoder
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
import torch

# load model
name_temp = 'sofa_ad'
model = ThreeDEPNDecoder()
model.load_state_dict(torch.load(f"runs/{name_temp}/model_best.ckpt", map_location='cpu'))
latent_vectors = torch.load(f"runs/{name_temp}/latent_best.pt", map_location = 'cpu')
log_vars = torch.load(f"runs/{name_temp}/log_var_best.pt", map_location = 'cpu')
Dist = dist.Normal(latent_vectors[1002], torch.exp(log_vars[1002]))
Dist2 = dist.Normal(latent_vectors[1000], torch.exp(log_vars[1000]))
#x_vad_n = torch.randn(256, device='cpu')
x_vad = Dist.rsample()#.unsqueeze(0)
x_vad2 = Dist2.rsample()#.unsqueeze(0)

a1 = 0.5
a2 = 0.5
f1 = 5
f2 = 105
output_meshes_int = model((x_vad*a1 + x_vad2*a2).unsqueeze(0))
output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

output_meshes = model(x_vad.unsqueeze(0))
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)

output_meshes = model(x_vad2.unsqueeze(0))
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)


Output()

Output()

Output()

In [62]:
from model.threedepn import ThreeDEPNDecoder
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
import torch
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = train_dataset[5]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)
# load model
name_temp = 'generalization_airplane_no_vad'
model = ThreeDEPNDecoder()
model.load_state_dict(torch.load(f"runs/{name_temp}/model_best.ckpt", map_location='cpu'))

# load latent codes and latent variances
latent_vectors = torch.load(f"runs/{name_temp}/latent_best.pt", map_location = 'cpu')
log_vars = torch.load(f"runs/{name_temp}/log_var_best.pt", map_location = 'cpu')

x_vad_n = x_vad_n.unsqueeze(0)
output_meshes = model(latent_vectors[5].unsqueeze(0))

# Visualize
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)

output_meshes = model(latent_vectors[105].unsqueeze(0))

# Visualize
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)
a1 = 0.5
a2 = 0.5
f1 = 5
f2 = 105
output_meshes_int = model((latent_vectors[f1]*a1 + latent_vectors[f2]*a2).unsqueeze(0))
output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

Target DF: (32, 32, 32)


Output()

Output()

Output()

Output()